# Integrate Agenta with OpenAI Agents SDK

This notebook demonstrates how to connect **Agenta** with **OpenAI Agents** for comprehensive observability and debugging of your multi-agent applications.

> **What is Agenta?** [Agenta](https://agenta.ai) is an open-source LLMOps platform designed to streamline the deployment, management, and scaling of large language models. It offers comprehensive observability, testing, and deployment capabilities for AI applications.

> **What is OpenAI Agents?** The [OpenAI Agents](https://github.com/openai/openai-agents-python) SDK is a framework for building AI agents that can interact with tools, run asynchronous operations, and orchestrate complex multi-agent workflows. It provides a structured approach to creating intelligent agents with tool calling capabilities.

## Implementation Guide

Follow this tutorial to set up OpenAI Agents SDK with Agenta's observability platform for real-time application insights.

### Step 1: Install Required Dependencies

Install the necessary Python packages for this integration:

In [ ]:
!pip install agenta openinference-instrumentation-openai-agents openai-agents

**Package Descriptions:**
- `agenta`: Core SDK for Agenta's prompt engineering and observability platform
- `openai-agents`: Framework for building multi-agent systems with tool calling capabilities
- `openinference-instrumentation-openai-agents`: Automatic instrumentation library for OpenAI Agents operations

### Step 2: Setup and Configuration

Configure your environment and initialize the Agenta SDK:

In [ ]:
import os
import asyncio
import agenta as ag
from agents import Agent, Runner
from openinference.instrumentation.openai_agents import OpenAIAgentsInstrumentor


# Load configuration from environment
os.environ["AGENTA_API_KEY"] = "your_agenta_api_key"
os.environ["AGENTA_HOST"] = (
    "https://cloud.agenta.ai"  # Optional, defaults to the Agenta cloud API
)
os.environ["OPENAI_API_KEY"] = "your_openai_api_key"  # Required for OpenAI Agents SDK


# Start Agenta SDK
ag.init()

**What does `ag.init()` do?**
This function initializes the Agenta SDK and sets up the necessary configuration for observability. It establishes connection to the Agenta platform, configures tracing and logging settings, and prepares the instrumentation context for your application.

### Step 3: Enable OpenAI Agents Monitoring

Initialize the OpenInference OpenAI Agents instrumentation to automatically capture agent operations:

In [ ]:
# Enable OpenAI Agents instrumentation
OpenAIAgentsInstrumentor().instrument()

### Step 4: Build Your Instrumented Multi-Agent Application

Here's a complete example showcasing a multi-agent translation system with Agenta instrumentation:

In [ ]:
# Define specialized translation agents
spanish_agent = Agent(
    name="Spanish agent",
    instructions="You translate the user's message to Spanish",
)

french_agent = Agent(
    name="French agent",
    instructions="You translate the user's message to French",
)

german_agent = Agent(
    name="German agent",
    instructions="You translate the user's message to German",
)

In [ ]:
# Create orchestrator agent with tool integration
orchestrator_agent = Agent(
    name="orchestrator_agent",
    instructions=(
        "You are a translation agent. You use the tools given to you to translate."
        "If asked for multiple translations, you call the relevant tools."
    ),
    tools=[
        spanish_agent.as_tool(
            tool_name="translate_to_spanish",
            tool_description="Translate the user's message to Spanish",
        ),
        french_agent.as_tool(
            tool_name="translate_to_french",
            tool_description="Translate the user's message to French",
        ),
        german_agent.as_tool(
            tool_name="translate_to_german",
            tool_description="Translate the user's message to German",
        ),
    ],
)

In [ ]:
@ag.instrument()
async def duolingo(query: str):
    result = await Runner.run(orchestrator_agent, input=query)
    return result.final_output

### Step 5: Run the Example

In [ ]:
# Example usage
async def main():
    response = await duolingo("What is 'What's your name' in French?")
    print("Response:", response)


# Run the example
await main()

### Step 6: Understanding the @ag.instrument() Decorator

The `@ag.instrument()` decorator automatically captures all input and output data from your function, enabling comprehensive observability without manual instrumentation.

**Span Type Configuration:**
Use the `spankind` parameter to categorize operations in Agenta WebUI. Available options:

- `agent` - Autonomous agent behaviors
- `chain` - Sequential processing workflows
- `workflow` - Complete application processes (default)
- `tool` - Utility and helper functions
- `embedding` - Vector embedding operations
- `query` - Search and retrieval tasks
- `completion` - Text generation operations
- `chat` - Conversational interfaces
- `rerank` - Result ordering operations

**Standard Behavior:**
By default, when `spankind` is not specified, the operation becomes a root-level span, categorized as a `workflow` in Agenta.

In [ ]:
# Example with custom span classification:
@ag.instrument(spankind="agent")
def specialized_agent_function(input_data: str):
    # Agent-specific logic implementation
    pass

### Step 7: View Traces in Agenta

After running your application, access detailed execution traces through Agenta's dashboard. The observability data includes:

- Complete multi-agent workflow execution timeline
- Agent initialization and configuration steps
- Tool calling and inter-agent communication
- Orchestrator decision-making processes
- Individual agent reasoning and response generation
- Performance metrics and timing analysis


<img 
    style="display: block; margin: 20px; text-align: center"
    src="./images/agenta-openinference-openai-agents-trace.png"
    width="90%"
    alt="Agenta dashboard showing OpenAI Agents SDK application trace with detailed execution steps">


The observability interface provides insights for:
- Debug complex multi-agent interactions and tool calling
- Monitor agent performance and decision-making patterns
- Analyze orchestration effectiveness and workflow optimization
- Track application behavior trends and identify bottlenecks

## Advanced Usage

### Custom Span Configuration

Customize instrumentation for different application components:

In [ ]:
@ag.instrument(spankind="workflow")
def multi_agent_pipeline(query: str):
    return asyncio.run(duolingo(query))


@ag.instrument(spankind="agent")
def custom_agent_handler(agent_input: str):
    # Custom agent logic
    pass


@ag.instrument(spankind="tool")
def external_tool_integration(tool_input: str):
    # External tool integration
    return tool_input

### Real-world Examples

#### Customer Support Agent System

In [ ]:
support_agent = Agent(
    name="Support Agent",
    instructions="You help customers with their inquiries and route complex issues to specialists",
)

billing_agent = Agent(
    name="Billing Agent",
    instructions="You handle billing-related questions and payment issues",
)

technical_agent = Agent(
    name="Technical Agent",
    instructions="You provide technical support and troubleshooting assistance",
)


@ag.instrument(spankind="workflow")
async def customer_support_system(customer_query: str):
    orchestrator = Agent(
        name="Support Orchestrator",
        instructions="Route customer queries to the appropriate specialist agent",
        tools=[
            support_agent.as_tool(
                "general_support", "Handle general customer inquiries"
            ),
            billing_agent.as_tool(
                "billing_support", "Handle billing and payment issues"
            ),
            technical_agent.as_tool(
                "technical_support", "Provide technical assistance"
            ),
        ],
    )

    result = await Runner.run(orchestrator, input=customer_query)
    return result.final_output

#### Research Analysis Team

In [ ]:
@ag.instrument(spankind="chain")
async def research_analysis_pipeline(research_topic: str):
    data_collector = Agent(
        name="Data Collector",
        instructions="Gather relevant information and data on the given topic",
    )

    analyst = Agent(
        name="Research Analyst",
        instructions="Analyze collected data and provide insights",
    )

    reporter = Agent(
        name="Report Writer",
        instructions="Create comprehensive reports based on analysis",
        tools=[
            data_collector.as_tool("collect_data", "Gather relevant research data"),
            analyst.as_tool("analyze_data", "Perform data analysis and insights"),
        ],
    )

    result = await Runner.run(reporter, input=research_topic)
    return result.final_output

#### Content Creation Workflow

In [ ]:
@ag.instrument(spankind="workflow")
async def content_creation_system(content_brief: str):
    writer = Agent(
        name="Content Writer",
        instructions="Create engaging content based on the brief",
    )

    editor = Agent(
        name="Content Editor",
        instructions="Review and improve content quality and style",
    )

    seo_specialist = Agent(
        name="SEO Specialist",
        instructions="Optimize content for search engines",
    )

    content_manager = Agent(
        name="Content Manager",
        instructions="Coordinate the content creation process and ensure quality",
        tools=[
            writer.as_tool("write_content", "Generate initial content draft"),
            editor.as_tool("edit_content", "Review and edit content"),
            seo_specialist.as_tool("optimize_seo", "Optimize content for SEO"),
        ],
    )

    result = await Runner.run(content_manager, input=content_brief)
    return result.final_output

#### Test the Advanced Examples

In [ ]:
# Test customer support system
support_response = await customer_support_system(
    "I have a billing question about my subscription"
)
print("Support Response:", support_response)

In [ ]:
# Test research analysis pipeline
research_response = await research_analysis_pipeline("Impact of AI on healthcare")
print("Research Response:", research_response)

In [ ]:
# Test content creation system
content_response = await content_creation_system(
    "Write a blog post about sustainable energy solutions"
)
print("Content Response:", content_response)

## Next Steps

For more detailed information about Agenta's observability features and advanced configuration options, visit the [Agenta Observability SDK Documentation](/observability/observability-sdk).